In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load Data

In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Replace with the actual file path in your Google Drive
file_path = '/content/drive/MyDrive/CMPE-256-Shared/data/combined_filtered_reviews.csv' #  Replace 'your_folder_name' and 'your_file.csv'

try:
  # Read the CSV file into a pandas DataFrame
  # typically it will cost around3 mins.
  df = pd.read_csv(file_path)

  # Print or process the DataFrame
  print("Loading data successfully")

except FileNotFoundError:
  print(f"Error: File not found at {file_path}. Please check the file path.")
except pd.errors.ParserError:
  print(f"Error: Could not parse the file at {file_path}.  Is it a valid CSV file?")
except Exception as e:
  print(f"An unexpected error occurred: {e}")

Mounted at /content/drive
Loading data successfully


# Preprocessing

In [ ]:
# prompt: 我希望统计author_id频率出现第一高和第二高的author_id，并打印出它们的频率。无需import，

author_id_counts = df['author_id'].value_counts()
top_two_author_ids = author_id_counts.nlargest(2)
top_two_author_ids

,count
author_id,
202,1193017
1504,23889


In [ ]:
author_id_mapping = {id: idx for idx, id in enumerate(df['author_id'].unique())}
hotel_id_mapping = {id: idx for idx, id in enumerate(df['hotel_id'].unique())}

df['author_id'] = df['author_id'].map(author_id_mapping)
df['hotel_id'] = df['hotel_id'].map(hotel_id_mapping)

In [ ]:
from datetime import datetime
TODAY = datetime.now()
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%dT%H:%M:%S")
df['date interval'] = (TODAY - df['date']).dt.days

In [ ]:
df.sample(1)

,date,rating,title,text,property_dict,hotel_id,author_id,date interval
2335950,2015-08-01,4.0,Nice downtown hotel,Overall a very good stay at the Hotel Palomar....,"{'service': 4.0, 'rooms': 4.0, 'cleanliness': ...",50836,6837,3410


In [ ]:
# prompt: 我想check hotel_id author_id rating 以及date interval这四栏是否有missing value

# Check for missing values in specified columns
missing_values = df[['hotel_id', 'author_id', 'rating', 'date interval']].isnull().sum()
missing_values

,0
hotel_id,0
author_id,0
rating,0
date interval,0


# Input Data preparation

In [ ]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device_name}")
DEVICE = torch.device(device_name)

Using device: cuda


In [ ]:
#DEVICE = torch.device("cpu")

In [ ]:
users = torch.tensor(df['author_id'].values, dtype=torch.long, device=DEVICE)
items = torch.tensor(df['hotel_id'].values, dtype=torch.long, device=DEVICE)
ratings = torch.tensor(df['rating'].values, dtype=torch.float32, device=DEVICE)
date_intervals = torch.tensor(df['date interval'].values, dtype=torch.long, device=DEVICE)

In [ ]:
num_users = len(author_id_mapping)
num_items = len(hotel_id_mapping)
num_factors = 6

# MF GPU version

## Data Split

In [ ]:
class RatingDataset(Dataset):
    def __init__(self, users, items, ratings):
        self.users = users
        self.items = items
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]


In [ ]:
# 划分数据
train_indices, test_indices = train_test_split(range(len(df)), test_size=0.2, random_state=42)

# 创建训练和测试数据集
train_dataset = RatingDataset(users[train_indices], items[train_indices], ratings[train_indices])
test_dataset = RatingDataset(users[test_indices], items[test_indices], ratings[test_indices])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)


In [ ]:
#for user, item, rating in train_loader:
#    assert user.is_cuda and item.is_cuda and rating.is_cuda, "Data is not on GPU"


## Model Definition

In [ ]:
class GPUMF(nn.Module):
    def __init__(self, num_users, num_items, num_factors):
        super(GPUMF, self).__init__()
        self.user_factors = nn.Embedding(num_users, num_factors)
        self.item_factors = nn.Embedding(num_items, num_factors)
        self.user_biases = nn.Embedding(num_users, 1)
        self.item_biases = nn.Embedding(num_items, 1)
        self.global_bias = nn.Parameter(torch.tensor(0.0))

        nn.init.xavier_uniform_(self.user_factors.weight)
        nn.init.xavier_uniform_(self.item_factors.weight)
        nn.init.zeros_(self.user_biases.weight)
        nn.init.zeros_(self.item_biases.weight)
        nn.init.zeros_(self.global_bias)

        self.device = DEVICE

        # 打印设备信息
        print(f"Model is running on {self.device}.")

        # 如果在 GPU 上运行，打印更多 GPU 信息
        if self.device.type == 'cuda':
            print(f"Running on {torch.cuda.get_device_name(torch.cuda.current_device())}")
            print(f"Total GPU Memory: {torch.cuda.get_device_properties(self.device).total_memory / 1e9:.2f} GB")
            print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(self.device) / 1e6:.2f} MB")
            print(f"GPU Memory Cached: {torch.cuda.memory_reserved(self.device) / 1e6:.2f} MB")

    def forward(self, user, item):
        user_embedding = self.user_factors(user)
        item_embedding = self.item_factors(item)
        user_b = self.user_biases(user).flatten()
        item_b = self.item_biases(item).flatten()
        preds = (user_embedding * item_embedding).sum(1) + user_b + item_b + self.global_bias
        return preds

    def fit(self, data_loader, epochs, lr, reg):
        self.to(self.device)
        self.train()
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            for user, item, rating in data_loader:
                user = user.to(self.device)
                item = item.to(self.device)
                rating = rating.to(self.device)
                optimizer.zero_grad()
                prediction = self(user, item)
                l2_reg = reg * (self.user_biases.weight.norm(2) + self.item_biases.weight.norm(2) + self.global_bias.norm(2))
                loss = criterion(prediction, rating) + l2_reg
                loss.backward()
                optimizer.step()
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    def predict(self, user, item):
        self.eval()
        with torch.no_grad():
            prediction = self(user, item)
            return prediction.item()


## Run

In [ ]:
model = GPUMF(num_users, num_items, num_factors)

Model is running on cuda.
Running on Tesla T4
Total GPU Memory: 15.84 GB
GPU Memory Allocated: 537.34 MB
GPU Memory Cached: 694.16 MB


In [ ]:
model.fit(train_loader, epochs=6, lr=0.0003, reg=0.02)

Epoch 1, Loss: 5.976485252380371
Epoch 2, Loss: 1.5466524362564087
Epoch 3, Loss: 1.1320040225982666
Epoch 4, Loss: 0.8018453121185303
Epoch 5, Loss: 0.8630028963088989
Epoch 6, Loss: 0.7495215535163879
Epoch 7, Loss: 0.869253396987915
Epoch 8, Loss: 0.9113022685050964


In [ ]:
def calculate_rmse(model, data_loader):
    model.eval()
    predictions, ratings = [], []
    with torch.no_grad():
        for user, item, rating in data_loader:
            prediction = model(user, item)
            predictions.append(prediction.cpu().numpy())
            ratings.append(rating.cpu().numpy())
    predictions = np.concatenate(predictions)
    ratings = np.concatenate(ratings)
    return np.sqrt(mean_squared_error(ratings, predictions))

In [ ]:
rmse = calculate_rmse(model, test_loader)
print(f"Test RMSE: {rmse}")

Test RMSE: 0.9037210941314697


# Time Decay MF

## Data Split

In [ ]:
# prompt: # prompt: 我要做一个比较复杂的data split，要求如下：针对每一个author_id，其对应了若干行。按照train size 0.8以及test size 0.2划分。但是划分不是随机的，而是根据date interval来划分。将date interval比较大的划分至train dataset，date interval比较小的划分至test dataset。无需import。最后返回的是对应的indices。使用groupby 似乎更快一些。

def split_data_by_date(df, train_size=0.8):
    train_indices = []
    test_indices = []

    for author_id, group in df.groupby('author_id'):
        group_sorted = group.sort_values('date interval', ascending=False)
        split_point = int(len(group_sorted) * train_size)
        train_indices.extend(group_sorted.index[:split_point].tolist())
        test_indices.extend(group_sorted.index[split_point:].tolist())

    return train_indices, test_indices

In [ ]:
class RatingDataset(Dataset):
    def __init__(self, users, items, ratings, timestamps):
        self.users = users
        self.items = items
        self.ratings = ratings
        self.date_intervals = date_intervals

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx], self.date_intervals[idx]


In [ ]:
# 划分数据
#train_indices, test_indices = train_test_split(range(len(df)), test_size=0.2, random_state=42)
train_indices, test_indices = split_data_by_date(df)


In [ ]:
# 创建训练和测试数据集
train_dataset = RatingDataset(users[train_indices], items[train_indices], ratings[train_indices], date_intervals[train_indices])
test_dataset = RatingDataset(users[test_indices], items[test_indices], ratings[test_indices], date_intervals[test_indices])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

## Model Definition

In [ ]:
# 定义时间衰减函数
def time_decay(date_interval, lamda):
    return torch.exp(-lamda * date_interval)

In [ ]:
# MF模型类
class TimeDecayMF(nn.Module):
    def __init__(self, num_users, num_items, num_factors):
        super(TimeDecayMF, self).__init__()
        self.user_factors = nn.Embedding(num_users, num_factors)
        self.item_factors = nn.Embedding(num_items, num_factors)
        self.user_biases = nn.Embedding(num_users, 1)
        self.item_biases = nn.Embedding(num_items, 1)
        self.global_bias = nn.Parameter(torch.tensor(0.0))

        nn.init.xavier_uniform_(self.user_factors.weight)
        nn.init.xavier_uniform_(self.item_factors.weight)
        nn.init.zeros_(self.user_biases.weight)
        nn.init.zeros_(self.item_biases.weight)
        nn.init.zeros_(self.global_bias)

        self.device = DEVICE

        # 打印设备信息
        print(f"Model is running on {self.device}.")

        # 如果在 GPU 上运行，打印更多 GPU 信息
        if self.device.type == 'cuda':
            print(f"Running on {torch.cuda.get_device_name(torch.cuda.current_device())}")
            print(f"Total GPU Memory: {torch.cuda.get_device_properties(self.device).total_memory / 1e9:.2f} GB")
            print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(self.device) / 1e6:.2f} MB")
            print(f"GPU Memory Cached: {torch.cuda.memory_reserved(self.device) / 1e6:.2f} MB")

    def forward(self, user, item):
        user_embedding = self.user_factors(user)
        item_embedding = self.item_factors(item)
        user_b = self.user_biases(user).flatten()
        item_b = self.item_biases(item).flatten()
        preds = (user_embedding * item_embedding).sum(1) + user_b + item_b + self.global_bias
        return preds

    # Fit 方法
    def fit(self, train_loader, epochs, lamda, lr, reg):
      self.to(self.device)
      self.train()
      optimizer = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=reg)

      for epoch in range(epochs):
          total_loss = 0

          for users, items, ratings, date_intervals in train_loader:
              optimizer.zero_grad()
              predictions = self(users, items)
              decay_weights = time_decay(date_intervals, lamda)
              loss = ((predictions - ratings) ** 2 * decay_weights).mean()
              loss.backward()
              optimizer.step()
              total_loss += loss.item()

          print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")

    def predict(self, user, item):
        self.eval()
        with torch.no_grad():
            prediction = self(user, item)
            return prediction.item()

## Run

In [ ]:
model = TimeDecayMF(num_users, num_items, num_factors)

Model is running on cuda.
Running on Tesla T4
Total GPU Memory: 15.84 GB
GPU Memory Allocated: 565.37 MB
GPU Memory Cached: 817.89 MB


In [ ]:
model.fit(train_loader, epochs=5, lamda=0.0002, lr=0.0003, reg=0.02)

Epoch 1/8, Loss: 5.312387053680018
Epoch 2/8, Loss: 1.8143493325900462
Epoch 3/8, Loss: 0.6301023703296253
Epoch 4/8, Loss: 0.5060127549402244
Epoch 5/8, Loss: 0.5057945774890383
Epoch 6/8, Loss: 0.5058017579219497
Epoch 7/8, Loss: 0.5057980230500894
Epoch 8/8, Loss: 0.5058123532810935


In [ ]:
def calculate_rmse(model, data_loader):
    model.eval()
    predictions, ratings = [], []
    with torch.no_grad():
        for user, item, rating, date_interval in data_loader:
            prediction = model(user, item)
            predictions.append(prediction.cpu().numpy())
            ratings.append(rating.cpu().numpy())
    predictions = np.concatenate(predictions)
    ratings = np.concatenate(ratings)
    return np.sqrt(mean_squared_error(ratings, predictions))

In [88]:
# 计算测试数据集上的 RMSE
test_rmse = calculate_rmse(model, test_loader)
print(f"Test RMSE: {test_rmse}")

Test RMSE: 1.0392874479293823
